In [1]:
from SimPy.Simulation import *
import random
import numpy as np

In [2]:
def skipHeader(dataFile):
    dataFile.readline()
    dataFile.readline()
    dataFile.readline()
    dataFile.readline()
    
def readData(periods):
    for i in range(1,11):
        f = open('./Data/results' + str(i) + '.txt', 'r')
        skipHeader(f) #Skips the four header lines at the time of the file.

        arrivals = [] # Array of arrivals for a period of time
        departures = [] # Array of departures for a period of time
        startTime = float(f.readline()[0:-1].split('  ')[0])
        endTime = None

        for line in f:
            observation = line[0:-1].split('  ')
            if observation[1] == 'a':
                arrivals.append(float(observation[0]))
            elif observation[1] == 'd':
                departures.append(float(observation[0]))
            else:
                endTime = float(observation[0])

        periods.append([startTime, arrivals, departures, endTime])

In [3]:
periods = [] # Array containing periods of format [start time, arrivals, departures, end time]
readData(periods)

interArrivalTimes = []
for p in periods:
    for a in range(1, len(p[1])):
        interArrivalTimes.append(p[1][a] - p[1][a - 1])        

'''serviceTimes = []
for p in periods:
    for d in range(1, len(p[2])):
        serviceTimes.append(p[2][d] - p[2][d - 1])'''

serviceTimes = []
for p in periods:
    serviceTimes.append(p[2][0]-p[1][0])
    for i in range(1,len(p[2])):
        if(p[2][i - 1] > p[1][i]):
            serviceTimes.append(p[2][i] - p[2][i - 1])
        else:
            serviceTimes.append(p[2][i] - p[1][i])

In [4]:
def draw_empirical(data, r):
    """one draw (for given r ~ U(0,1)) from the
    empirical cdf based on data"""
    
    d = {x:data.count(x) for x in data}
    obs_values, freq = zip( *sorted( zip(d.keys(), d.values())))
    obs_values = list(obs_values)
    freq = list(freq)
    empf = [x*1.0/len(data) for x in freq]
    ecum = np.cumsum(empf).tolist()
    ecum.insert(0, 0)
    obs_values.insert(0,0)
   
    for x in ecum:
         if r <= x:
            rpt = x
            break
    r_end = ecum.index(rpt)
    y = obs_values[r_end] - 1.0*(ecum[r_end]-r)*(obs_values[r_end]-
        obs_values[r_end-1])/(ecum[r_end]-ecum[r_end-1])
    return y

In [5]:
## Model ----------
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = draw_empirical(interArrivalTimes,random.random())
            yield hold, self, t

class Arrival(Process):
    n = 0 # number in system
    q = 0 # number in queue
    
    """an arrival"""
    def run(self):
        Arrival.n += 1
        Arrival.q += 1
        arrivetime = now()
        G.arrivalmon.observe(now())
        G.numbermon.observe(Arrival.n)
        G.qnumbermon.observe(Arrival.q)
        
        yield request, self, G.server
        Arrival.q += -1
        G.qnumbermon.observe(Arrival.q)
        G.queuetimemon.observe(now() - arrivetime)
        t = draw_empirical(serviceTimes,random.random())
        G.servicemon.observe(t)
        G.servicesquaredmon.observe(t**2)
        yield hold, self, t
        yield release, self, G.server
        
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        delay = now()-arrivetime
        G.delaymon.observe(delay)
        #print now(), "Observed delay", delay


class G:
    server = 'dummy'
    delaymon = 'Monitor'
    servicemon = 'Monitor'
    servicesquaredmon = 'Monitor'
    numbermon = 'Monitor'
    arrivalmon = 'Monitor'
    departuremon = 'Monitor'
    queuetimemon = 'Monitor'
    
def model(c, N, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c, monitored=True)
    G.delaymon = Monitor()
    G.servicemon = Monitor()
    G.servicesquaredmon = Monitor()
    G.numbermon = Monitor()
    G.arrivalmon = Monitor()
    G.departuremon = Monitor()
    G.queuetimemon = Monitor()
    G.qnumbermon = Monitor()
    
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)
   
    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    Lq = G.qnumbermon.timeAverage()
    S = G.servicemon.mean()
    Svar = G.servicemon.var()
    S2 = G.servicesquaredmon.mean()
    util = G.server.actMon.timeAverage()
    Wq = G.queuetimemon.mean()
    
    # interarrival mean and variance
    interarrivals = []
    arrivals = G.arrivalmon.tseries()
    for a in range(1,len(arrivals)):
        interarrivals.append(arrivals[a] - arrivals[a - 1])
    IA = np.mean(interarrivals)
    IAvar = np.var(interarrivals)
    
    return(W,S,S2,L,util,IA,Wq,Lq,IAvar,Svar)

## Experiment ----------------
allW = []
allWq = []
allS = []
allL = []
allLq = []
allUtil = []
allIA = []
allIAvar = []
allSvar = []

for i in range(50):
    if i == 9:
        print("Doing", str(i + 1) + "th", "iteration")
    elif i == 19:
        print("Doing", str(i + 1) + "th", "iteration")
    elif i == 29:
        print("Doing", str(i + 1) + "th", "iteration")
    elif i ==39:
        print("Doing", str(i + 1) + "th", "iteration")
        
    result = model(c=1, N=30, maxtime=2000000000, rvseed=123*i)
    
    allW.append(result[0])
    allS.append(result[1])
    allL.append(result[3])
    allUtil.append(result[4])
    allIA.append(result[5])
    allWq.append(result[6])
    allLq.append(result[7])
    allIAvar.append(result[8])
    allSvar.append(result[9])

print("")
print("ESTIMATES")
print("---------")
print("Interarrival mean:", np.mean(allIA))
print("W:", np.mean(allW))
print("Wq:", np.mean(allWq))
print("Ws:", np.mean(allS))
print("L:", np.mean(allL))
print("Lq:", np.mean(allLq))
print("Utilisation:", np.mean(allUtil))
print("Interarrival variance:", np.mean(allIAvar))
print("Ws variance:", np.mean(allSvar))

Doing 10th iteration
Doing 20th iteration
Doing 30th iteration
Doing 40th iteration

ESTIMATES
---------
Interarrival mean: 57.82270086610693
W: 101.16207990919129
Wq: 59.0126451487929
Ws: 42.14943476039837
L: 1.7974919559019682
Lq: 1.0762346460714023
Utilisation: 0.7212573098305658
Interarrival variance: 4111.924484301638
Ws variance: 554.7860662397843


In [ ]:
serviceTimes = G.wsMon.yseries()
interArrivalTimes = G.interarrivalmon.yseries()

serviceBins = int(round(1 + np.log2(len(serviceTimes))))
interArrivalBins = int(round(1 + np.log2(len(interArrivalTimes))))

In [ ]:
#Service Histogram
plt.hist(x=serviceTimes, bins=serviceBins, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Service Time (s)')
plt.ylabel('Frequency')
plt.title('Service Histogram')
plt.plot();

print("Service Times:\n")
print(serviceTimes)

In [ ]:
#Interarrival Histogram
plt.hist(x=interArrivalTimes, bins=interArrivalBins, color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Interarrival Time (s)')
plt.ylabel('Frequency')
plt.title('Interarrival Histogram')
plt.plot();

print("interarival times:\n")
print(interArrivalTimes)